<a href="https://colab.research.google.com/github/McSloats/PAM-Backdoor/blob/master/Indep_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import numpy as np
import csv
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers import Dense
from sklearn import preprocessing
from keras import optimizers as op
from keras.utils import to_categorical
from keras.models import model_from_json
from keras.callbacks import EarlyStopping


Using TensorFlow backend.


In [10]:
##IMPORT FILE FOR DATA EXTRACTION
benign = pd.read_csv("/content/drive/My Drive/ben.csv", header=None, dtype=str)
malicious = pd.read_csv("/content/drive/My Drive/mal.csv", header=None, dtype=str)
#Labels
benign['label'] = 1
malicious['label'] = 0

#Combine
data = [benign,malicious]
data = pd.concat(data)

#Shuffle
data = data.sample(frac=0.080,random_state=1).reset_index(drop=True)

#Remove Labels
Y = data.iloc[:,-1]
X = data.iloc[:,:-1]

print(X)
print(Y)

        0   1   2   3   4   5   6   7   8   ...  45  46  47  48  49  50  51  52  53
0       08  00  27  b5  b7  19  00  1e  49  ...  fa  50  18  fa  9a  81  64  00  00
1       52  54  00  12  35  02  08  00  27  ...  02  50  18  50  00  cc  ab  00  00
2       02  1a  c5  01  00  00  02  1a  c5  ...  1f  80  10  21  f0  23  dc  00  00
3       02  1a  c5  01  00  00  02  1a  c5  ...  f9  80  10  21  f0  17  0e  00  00
4       ff  ff  ff  ff  ff  ff  08  00  27  ...  00  00  00  00  00  00  00  00  00
...     ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ..  ..  ..  ..  ..  ..  ..  ..  ..
725358  00  1e  49  db  19  c3  08  00  27  ...  ee  50  10  d8  b8  e3  e3  00  00
725359  ff  ff  ff  ff  ff  ff  08  00  27  ...  00  00  00  00  00  00  00  00  00
725360  02  1a  c5  01  00  00  02  1a  c5  ...  f2  80  18  49  88  4f  2e  00  00
725361  08  00  27  fb  bb  39  52  54  00  ...  f9  50  18  ff  ff  88  cf  00  00
725362  08  00  27  fb  bb  39  52  54  00  ...  ad  50  10  ff  ff  a1  ec 

In [11]:
#Create dictionary
dictionary = ['08', '00', '27', 'b5', 'b7', '19', '1e', '49', 'db', 'c3', '45', '2d', '34', '29', '6c', '06', 'f5', 'd7', 'b8', '9a', '84', '6a', '93', '20', '54', 'a5', '25', '05', '76', 'd9', 'fd', 'd1', '4c', '40', 'ab', 'fa', '50', '18', '81', '64', '52', '12', '35', '02', 'fb', 'bb', '39', '01', '33', '2f', '13', '80', '07', '36', '0a', '5d', 'bd', '59', '53', 'e6', 'e9', '1f', '90', '83', '61', '43', '1a', '22', 'cc', 'c5', 'dc', '5c', '9e', '79', '60', '0f', '24', '0c', '85', '10', '21', 'f0', '23', '7a', 'ef', '55', '92', '7d', '1b', 'af', '7e', 'd2', 'cd', '9f', 'f9', '17', '0e', 'ff', 'c1', '4e', 'e4', '11', 'cb', '51', '6b', '89', '3a', '4f', 'ba', '30', '2c', '75', 'f7', 'b2', '4d', '47', '31', '0d', '3d', '5e', '26', '94', '2e', 'e3', 'b0', 'fe', 'f6', '9d', 'e2', '09', '8f', '74', '7b', 'bf', 'd4', 'b1', 'bc', '8c', 'de', 'd8', '46', '0b', '82', 'b6', '3c', 'ca', '71', 'a9', '14', '8d', '98', 'e0', 'd0', 'a6', '78', '28', 'ea', 'cf', '41', '37', 'c4', 'd6', 'aa', '58', '6f', 'c0', '8a', '04', 'f8', '70', 'be', 'a7', '65', 'a4', '87', 'ed', 'c7', '68', '5b', '16', '9b', '57', 'a3', '38', '88', '9c', 'a8', '91', '69', '73', '2b', 'a0', 'ec', 'ad', '44', 'eb', '3e', '5f', '3f', 'e8', '32', '1d', '48', 'fc', '42', '8e', 'f1', '6e', '62', '1c', 'ae', '7c', 'a2', '03', 'ce', '15', 'd3', '86', 'e1', '56', '6d', '4a', '77', 'da', '67', 'e7', 'f4', 'c9', '7f', '66', 'c2', '3b', '4b', '8b', '72', '99', 'c8', 'b4', '95', '96', 'ac', 'b9', '2a', 'dd', 'df', 'c6', '97', 'f3', 'a1', '5a', 'f2', '63', 'e5', 'ee', 'b3', 'd5']
data = []
for row in range(0,X.shape[0]):
  tmp = []
  for col in range(0,X.shape[1]):
    word = X.iloc[row][col]
    tmp.append(dictionary.index(word))
  data.append(tmp)
data = np.array(data)
print(dictionary)
print(data)

['08', '00', '27', 'b5', 'b7', '19', '1e', '49', 'db', 'c3', '45', '2d', '34', '29', '6c', '06', 'f5', 'd7', 'b8', '9a', '84', '6a', '93', '20', '54', 'a5', '25', '05', '76', 'd9', 'fd', 'd1', '4c', '40', 'ab', 'fa', '50', '18', '81', '64', '52', '12', '35', '02', 'fb', 'bb', '39', '01', '33', '2f', '13', '80', '07', '36', '0a', '5d', 'bd', '59', '53', 'e6', 'e9', '1f', '90', '83', '61', '43', '1a', '22', 'cc', 'c5', 'dc', '5c', '9e', '79', '60', '0f', '24', '0c', '85', '10', '21', 'f0', '23', '7a', 'ef', '55', '92', '7d', '1b', 'af', '7e', 'd2', 'cd', '9f', 'f9', '17', '0e', 'ff', 'c1', '4e', 'e4', '11', 'cb', '51', '6b', '89', '3a', '4f', 'ba', '30', '2c', '75', 'f7', 'b2', '4d', '47', '31', '0d', '3d', '5e', '26', '94', '2e', 'e3', 'b0', 'fe', 'f6', '9d', 'e2', '09', '8f', '74', '7b', 'bf', 'd4', 'b1', 'bc', '8c', 'de', 'd8', '46', '0b', '82', 'b6', '3c', 'ca', '71', 'a9', '14', '8d', '98', 'e0', 'd0', 'a6', '78', '28', 'ea', 'cf', '41', '37', 'c4', 'd6', 'aa', '58', '6f', 'c0', '8a

In [13]:
with open('/content/data.txt', 'w') as f:
    for item in data:
        f.write(item)

TypeError: ignored

In [0]:
temp_data = pd.DataFrame(data)
temp_data = temp_data.iloc[:,[12,13,14,15,16,17,18,19,20,21,22,23,24,25,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53]]

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(temp_data, Y, test_size=0.2)
y_test = to_categorical(y_test)
y_train = to_categorical(y_train)

In [21]:
print(temp_data)

        12  13  14  15  16   17   18   19  ...  46  47   48   49   50   51  52  53
0        0   1  10   1   1   11   12   13  ...  36  37   35   19   38   39   1   1
1        0   1  10   1  47   48   49   50  ...  36  37   36    1   68   34   1   1
2        0   1  10   1  27   70   71   72  ...  51  79   80   81   82   70   1   1
3        0   1  10   1  27   70   83   84  ...  51  79   80   81   95   96   1   1
4        0   1  10   1   1   99   85  100  ...   1   1    1    1    1    1   1   1
...     ..  ..  ..  ..  ..  ...  ...  ...  ...  ..  ..  ...  ...  ...  ...  ..  ..
725358   0   1  10   1   1  155  215  171  ...  36  79  139   18  123  123   1   1
725359   0   1  10   1   1   99   14   48  ...   1   1    1    1    1    1   1   1
725360   0   1  10   1  27   70   72   94  ...  51  37    7  184  107  122   1   1
725361   0   1  10   1  27  148  106  188  ...  36  37   97   97  184  157   1   1
725362   0   1  10   1   1  155  117  234  ...  36  79   97   97  248  192   1   1

[72

In [0]:
def rnn(x_train, y_train, x_test, y_test, dictionary):
    #KERAS MODEL    
    #create model
    model = Sequential()
    #get number of columns in training data
    n_cols = x_train.shape[1]
    #add model layers
    model.add(Embedding(len(dictionary), 64,input_length=n_cols))
    model.add(LSTM(128,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(64,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(32))
    model.add(Dropout(0.2))
    model.add(Dense(2, activation='softmax'))
   
    #Optimizer
    opt = op.Adam()
    #Compile model
    model.compile(opt, loss='binary_crossentropy', 
                  metrics=['accuracy'])
   
    #TRAINING
    #Put this var in callbacks to save model after each epoch
    #checkpointer = ModelCheckpoint('model/model-{epoch:02d}.hdf5', verbose=1)

    model.fit(x_train, y_train, batch_size=256, 
              validation_split=0.2, epochs=5)

    #test results
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

    return model

In [33]:
model = rnn(x_train, y_train, x_test, y_test, dictionary)


Train on 464232 samples, validate on 116058 samples
Epoch 1/5
464232/464232 [==============================] - 484s 1ms/step - loss: 0.0127 - acc: 0.9946 - val_loss: 5.0019e-04 - val_acc: 0.9998
Epoch 2/5
464232/464232 [==============================] - 481s 1ms/step - loss: 2.8452e-04 - acc: 0.9999 - val_loss: 6.2666e-05 - val_acc: 1.0000
Epoch 3/5
464232/464232 [==============================] - 479s 1ms/step - loss: 2.6313e-05 - acc: 1.0000 - val_loss: 2.3664e-06 - val_acc: 1.0000
Epoch 4/5
464232/464232 [==============================] - 482s 1ms/step - loss: 2.5038e-04 - acc: 0.9999 - val_loss: 1.5125e-04 - val_acc: 1.0000
Epoch 5/5
464232/464232 [==============================] - 481s 1ms/step - loss: 6.2934e-05 - acc: 1.0000 - val_loss: 5.7335e-07 - val_acc: 1.0000
Test loss: 3.5069545666505397e-07
Test accuracy: 1.0


In [0]:
model_json = model.to_json()
with open("/content/model.json","w") as json_file:
  json_file.write(model_json)
model.save_weights("/content/model.h5")